# Analysing historical records of wave data from IMOS

Access to historical records is important to 
* evaluate global and local impacts of climate change & anthropogenic influences and 
* test and validate models. In the modelling area, this kind of approach is done  routinely to evaluate models performances (hindcast models).

Here we will see how we can query historical data from a buoy located offshore Sydney. 

Our dataset is available from: 

<a href='http://imos.org.au'>IMOS Data Portal</a> and has been recorded by the <a href='http://new.mhl.nsw.gov.au/TodaysConditions-Sydney'>**Manly Hydraulics Laboratory**</a>:
+ <a href='http://imos-data.s3-website-ap-southeast-2.amazonaws.com/?prefix=NSW-OEH/Manly_Hydraulics_Laboratory/Wave/Sydney/'>dataset link</a>

We will use the **netCDF** libary (<a ref='http://www.unidata.ucar.edu/netcdf/'>Network Common Data Form</a>) is a set of software libraries and self-describing, machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. The project homepage is hosted by the **Unidata** program at the University Corporation for Atmospheric Research (**UCAR**).

Loading a module is straight forward:

In [ ]:
%matplotlib inline
from pylab import *
import netCDF4
import datetime as dt
import numpy as np
import pandas as pd

from pylab import rcParams
import warnings
warnings.filterwarnings('ignore')

# 2015 Waverider Buoy dataset

We will use a dataset containing ocean wave data from wave monitoring buoy moored off Sydney at latitude 33$^o$46'26" S, longitude 151$^o$24'42" E and water depth 90 metres.  

<a href='https://s3-ap-southeast-2.amazonaws.com/imos-data/NSW-OEH/Manly_Hydraulics_Laboratory/Wave/Sydney/Sydney_Waverider_Buoy_location_history.pdf'> Sydney Waverider Buoy location history</a>



The data is gathered using the Directional Waverider system developed by the Dutch company, Datawell. The Directional Waverider buoy utilises a heave-pitch-roll sensor, two fixed X and Y accelerometers and a three axis fluxgate compass to measure both vertical and horizontal motion.  An on-board processor converts the buoy motion to three orthogonal (vertical, north-south, east-west) translation signals that are transmitted to the shore station.  The directional spectrum is also routinely transmitted to the receiving station for further processing.  The wave data is stored on the receiving station PC before routine transfer to Manly Hydraulics Laboratory.

In [ ]:
# Offshore Sydney buoy data
filename = '../../dataset/IMOS_ANMN-NSW_W_20150210T230000Z_WAVESYD_WAVERIDER_FV01_END-20171231T130000Z.nc'
# Offshore Batemans Bay buoy data
#filename = '../../dataset/IMOS_ANMN-NSW_W_20080124T210000Z_WAVEBAB_WAVERIDER_FV01_END-20151231T130000Z.nc'
nc_data=netCDF4.Dataset(filename)

## Query dataset

Let have a look at the loaded netCDF variables

In [ ]:
nc_data.variables.keys()

In [ ]:
print(nc_data.variables['TIME'])

## Find out time interval record

Get the time extension of the gathered data...

In [ ]:
units = nc_data.variables['TIME'].units
calendar = nc_data.variables['TIME'].calendar

times = netCDF4.num2date(nc_data.variables['TIME'][:], units=units, calendar=calendar)

In [ ]:
start = dt.datetime(1985,1,1)
# Get desired time step  
time_var = nc_data.variables['TIME']
itime = netCDF4.date2index(start,time_var,select='nearest')
dtime = netCDF4.num2date(time_var[itime],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')

print('buoy record start time:',daystr)

In [ ]:
end = dt.datetime(2018,1,1)
# Get desired time step  
time_var = nc_data.variables['TIME']
itime2 = netCDF4.date2index(end,time_var,select='nearest')
dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
dayend = dtime2.strftime('%Y-%b-%d %H:%M')

print('buoy record end time:',dayend)

In [ ]:
print('Records per day:')
for k in range(1,25):
    print(netCDF4.num2date(time_var[k],time_var.units))

## Buoy location

Check the location of the data

In [ ]:
loni = nc_data.variables['LONGITUDE'][:]
lati = nc_data.variables['LATITUDE'][:]
print(loni,lati)
names=[]
names.append('Offshore Sydney Buoy')

## Visualise buoy records

Export for the historical time serie, the desired buoy variables, here we use HRMS:

In [ ]:
times = nc_data.variables['TIME']
jd_data = netCDF4.num2date(times[:],times.units, only_use_cftime_datetimes=False, 
                           only_use_python_datetimes=True).flatten()
hm_data = nc_data.variables['HRMS'][:].flatten()
#T_data = ???

In [ ]:
jd_data

Now plot the exported dataset

In [ ]:
MyDateFormatter = DateFormatter('%Y-%b-%d')
fig = plt.figure(figsize=(16,10), dpi=80) 
ax1 = fig.add_subplot(111)

ax1.plot(jd_data[itime:itime2],hm_data[itime:itime2],linewidth=1) 
ax1.xaxis.set_major_locator(WeekdayLocator(byweekday=MO,interval=12))
ax1.xaxis.set_major_formatter(MyDateFormatter)
ax1.grid(True)
setp(gca().get_xticklabels(), rotation=45, horizontalalignment='right')
plt.title('IMOS Offshore Sydney Buoy Data since 2015')
ax1.set_ylabel('meters')
ax1.legend(names,loc='upper right')

# April 2015 storm

![alt text](https://i.guim.co.uk/img/media/643dfa81d4bbca129906d8a26c683e3e6901c053/0_110_3000_1801/3000.jpg?w=1010&q=55&auto=format&usm=12&fit=max&s=6dac931858af8c21404a5604900ae71d)

_A man fools around in the wind at Bondi Beach, Sydney, on April 21 2015._

To find the temporal `id` of the storm we use the python function `argmax`:

In [ ]:
idmax = hm_data[itime:itime2].argmax()
print('Temporal ID of the strom:',idmax)
print('Corresponding date:',jd_data[idmax])

## Plotting variables 

Set the wave height RMS 48h before and after the recorded maximum wave height RMS time.

In [ ]:
storm_time = ???
storm_hrms = ???
storm_period = ???

In [ ]:
MyDateFormatter = DateFormatter('%b-%d %H:%M')
fig = plt.figure(figsize=(10,10), dpi=80) 
ax1 = fig.add_subplot(111)

ax1.plot(storm_time,storm_hrms,linewidth=4) 
ax1.xaxis.set_major_locator(HourLocator(byhour=range(24),interval=6))
ax1.xaxis.set_major_formatter(MyDateFormatter)
ax1.grid(True)
setp(gca().get_xticklabels(), rotation=45, horizontalalignment='right')
plt.title('Offshore Sydney Buoy Data for April 2015 storm')
ax1.set_ylabel('meters')
names = ['HRMS']
ax1.legend(names,loc='upper right')

## Wave power and energy

- Wave power:
$$P = \frac{\rho g^2}{64 \pi} H_{mo}^2 T \simeq 0.5 H_{mo}^2 T$$

- Wave energy:
$$E = \frac{1}{16}\rho g H_{mo}^2$$

_Note:_
You can define $\pi$ using np.pi.

### Define the functions

In [ ]:
def wave_power(rho,g,h,t):
    power = ???
    return power

def wave_energy(rho,g,h):
    energy = ???
    return energy

In [ ]:
storm_power = wave_power(???)
storm_energy = wave_energy(???)

### Plot the data over the storm duration

In [ ]:
dates = pd.to_datetime(data_df['TIME'], format = '%Y-%m-%dT%H:%M:%SZ')

In [ ]:
data_df['WHTH']

In [ ]:
import matplotlib.dates as mdates
MyDateFormatter = DateFormatter('%Y-%b-%d')
fig = plt.figure(figsize=(16,10), dpi=160) 
ax1 = fig.add_subplot(111)
ax1.plot(dates,data_df['WHTH'],linewidth=3) 
locator = mdates.AutoDateLocator()
ax1.xaxis.set_major_locator(locator)
ax1.xaxis.set_major_formatter(MyDateFormatter)
ax1.grid(True)
setp(gca().get_xticklabels(), rotation=45, horizontalalignment='right')
plt.title('IMOS Offshore Sydney Buoy Data since 2015')
ax1.set_ylabel('meters')

fig.show()